<h1>Regressie</h1>
begin met importeren van databases in een dataframe

In [1]:
import pandas as pd
import pyodbc

# Maak verbinding met de Microsoft Access-database
conn_str_sales = (
    r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=C:\\Users\\Burto\\Downloads\\GO-databases DEDS-week 4 & 5\\HC code\Week 10\\GO-databases schoon\\go_sales_schoon.accdb;"
    r"ExtendedAnsiSQL=1;"
)

conn_SALES = pyodbc.connect(conn_str_sales)

SALESdf = pyodbc.connect(conn_str_sales)

<h2>Maak de juiste dataframes</h2>

In [2]:
df_returnedItems = pd.read_sql_query("SELECT * FROM returned_item", conn_SALES)
df_details = pd.read_sql_query("SELECT * FROM order_details", conn_SALES)
df_header = pd.read_sql_query('SELECT * FROM order_header', conn_SALES)

#JOIN tabel
df = pd.merge(df_returnedItems, df_details, on='ORDER_DETAIL_CODE')
df = pd.merge(df,df_header, on='ORDER_NUMBER')

#Selecteer alleen de relevante informatie
df = df.dropna()
df = df[['ORDER_DATE','RETURN_QUANTITY']]

C:\Users\Burto\AppData\Local\Temp\ipykernel_18692\2438351441.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_returnedItems = pd.read_sql_query("SELECT * FROM returned_item", conn_SALES)
C:\Users\Burto\AppData\Local\Temp\ipykernel_18692\2438351441.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_details = pd.read_sql_query("SELECT * FROM order_details", conn_SALES)
C:\Users\Burto\AppData\Local\Temp\ipykernel_18692\2438351441.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_header = pd.read_sql_query('SELEC

<h2>Regressiemodel</h2>
<p>We kunnen beginnen met het toevoegen van een nieuwe kolom met het jaartal van de bestelling en het verwijderen van de ORDER_DATE-kolom</p>

In [3]:
df['YEAR'] = pd.DatetimeIndex(df['ORDER_DATE']).year


<p>vervolgens kunnen we de variabelen opsplitsen in afhankelijke en onafhankelijke variabelen. In dit geval is de RETURN_QUANTITY de afhankelijke variabele en DAY_OF_WEEK de onafhankelijke variabele:</p>

In [4]:
X = df[['YEAR']]
y = df['RETURN_QUANTITY']


Nu kunnen we een lineaire regressie model maken door de LinearRegression klasse te gebruiken uit Scikit-learn:

In [5]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X=X, y=y, sample_weight=None)


LinearRegression()

</p>We kunnen nu het model gebruiken om voorspellingen te doen op nieuwe data. We kunnen bijvoorbeeld het aantal retourzendingen voorspellen voor de dinsdag:</p>

In [9]:
predicted_returns = model.predict([[2024]])
print(predicted_returns)


[20.95152058]


C:\Users\Burto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


<h3>Bereken R2 score</h3>
<p>Een hogere score is beter.<br/>Laten we de code testen door 2024 in te vullen, kijken wat er uit komt.</p>

In [10]:
from sklearn.metrics import r2_score

y_pred = model.predict(X)
r2 = r2_score(y, y_pred)
print('R2-score:', r2)


R2-score: 2.2947381396853928e-05


<h2>Polynomial poging</h2>

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

model_poly = LinearRegression()
model_poly.fit(X_poly, y)
